In [1]:
import seaborn as sns
from fastai.imports import *
from glob import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from fastai.structured import *
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
%matplotlib inline


In [2]:
from tsfresh import select_features, extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.distribution import MultiprocessingDistributor

In [3]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
def evaluate(model, test_features, test_labels):
    
    accuracy = m.score(test_features, test_labels)
    mse = mean_squared_error(test_labels, model.predict(test_features))
    
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(accuracy))
    print('Mean Squared Error = {:0.4f}%.'.format(mse))
    return accuracy

In [4]:
PATH = '/media/joanes/0BB3-1FA1/CSV_DATA/'
files = glob(f'{PATH}*.csv')
# Get only those values that are available before the analysis
filter_cols = ['Total_UnfilledZones', 'Total_FillingQuality', 'TOTAL_QUALITY',  'Total_PorosityQuantity', 'Total_PorosityQuality']
df_list = list()
target = list()
for id, file in enumerate(files[:10]):
    df = pd.read_csv(file)
    target.append(df.TOTAL_QUALITY.unique()[0])
    df_filtered = df.drop(axis=1, columns=filter_cols)
    df_filtered['id'] = id
    filter_col = [col for col in df_filtered if not col.endswith(('VoidContent','VoidQuality', 'Filling', 'FillingQuality'))]
    df_list.append(df_filtered[filter_col])

In [8]:
huge_df = pd.concat(df_list)
huge_df.id.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:

Distributor = MultiprocessingDistributor(n_workers=8,disable_progressbar=False,progressbar_title="Feature Extraction")


extracted_features = extract_features(huge_df, column_id='id', distributor=Distributor)

Feature Extraction:   0%|          | 0/40 [00:00<?, ?it/s]/home/joanes/miniconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1453: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/home/joanes/miniconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1454: RuntimeWarning: divide by zero encountered in log
  se = -1 * np.log(similarity_ratio)
/home/joanes/miniconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1496: RuntimeWarning: invalid value encountered in double_scalars
  return sum_product / ((len(x) - lag) * np.var(x))
/home/joanes/miniconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1453: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:773: RuntimeWarning: invalid value encountered in double_scala

/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:846: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:846: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:846: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/joanes/minic

/home/joanes/miniconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1496: RuntimeWarning: invalid value encountered in double_scalars
  return sum_product / ((len(x) - lag) * np.var(x))
/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:773: RuntimeWarning: invalid value encountered in double_scalars
  phi[1, 1] = sxx_m[1] / sxx_m[0]
/home/joanes/miniconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
Feature Extraction:   5%|▌         | 2/40 [02:51<1:15:45, 119.62s/it]/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:777: RuntimeWarning: invalid 

/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:773: RuntimeWarning: invalid value encountered in double_scalars
  phi[1, 1] = sxx_m[1] / sxx_m[0]
/home/joanes/miniconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
Feature Extraction:  35%|███▌      | 14/40 [10:01<19:51, 45.84s/it]/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:846: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encoun

/home/joanes/miniconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1453: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/home/joanes/miniconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1454: RuntimeWarning: divide by zero encountered in log
  se = -1 * np.log(similarity_ratio)
/home/joanes/miniconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1496: RuntimeWarning: invalid value encountered in double_scalars
  return sum_product / ((len(x) - lag) * np.var(x))
/home/joanes/miniconda3/lib/python3.6/site-packages/statsmodels/tsa/stattools.py:773: RuntimeWarning: invalid value encountered in double_scalars
  phi[1, 1] = sxx_m[1] / sxx_m[0]
/home/joanes/miniconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In

In [7]:
extracted_features.head()

variable,Flow rate__abs_energy,Flow rate__absolute_sum_of_changes,"Flow rate__agg_autocorrelation__f_agg_""mean""","Flow rate__agg_autocorrelation__f_agg_""median""","Flow rate__agg_autocorrelation__f_agg_""var""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","Flow rate__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,Zone_ids__time_reversal_asymmetry_statistic__lag_1,Zone_ids__time_reversal_asymmetry_statistic__lag_2,Zone_ids__time_reversal_asymmetry_statistic__lag_3,Zone_ids__value_count__value_-inf,Zone_ids__value_count__value_0,Zone_ids__value_count__value_1,Zone_ids__value_count__value_inf,Zone_ids__value_count__value_nan,Zone_ids__variance,Zone_ids__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
0,1.099151e-07,0.000050,0.0,0.0,0.0,0.000012,-0.196922,-3.750376e-08,2.049529e-08,0.000013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.054345e-07,0.000054,0.0,0.0,0.0,0.000013,-0.317928,-6.308345e-08,2.028611e-08,0.000014,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.114102e-07,0.000055,0.0,0.0,0.0,0.000013,-0.322060,-6.484049e-08,2.043485e-08,0.000014,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.087021e-07,0.000060,0.0,0.0,0.0,0.000013,-0.371239,-6.937613e-08,1.849758e-08,0.000014,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.004678e-07,0.000070,0.0,0.0,0.0,0.000013,-0.355572,-6.437397e-08,1.793642e-08,0.000014,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Store featurized dataframe to feather files

In [ ]:
import os 
os.makedirs('tmp', exist_ok=True)
test.to_feather('tmp/test')
train.to_feather('tmp/train')
pd.DataFrame(target, columns=['valid']).to_feather('tmp/target')

## Read feather file into dataframe

In [ ]:
import feather

test = feather.read_dataframe('tmp/test')
train = feather.read_dataframe('tmp/train')
target =feather.read_dataframe('tmp/target')

In [ ]:
train.shape, test.shape, target.shape

## Get initial classifier's score

In [ ]:
m = RandomForestClassifier(n_estimators=80,max_features=0.5, min_samples_leaf=10, oob_score=True, n_jobs=-1)
m.fit(train, target[:2000].values.ravel())
print(f'Training score: {m.score(train, target[:2000].values.ravel())}')
print(f'Testing score: {m.score(test, target[2000:].values.ravel())}')
print(f'Out of bag score: {m.oob_score_}')
print('\n')
evaluate(m, test, target[2000:]);

## Get classifier's confusion matrix

In [ ]:
y_test = m.predict(test)

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, target[2000:])

# Plot non-normalized confusion matrix

plot_confusion_matrix(cnf_matrix, classes=['Egokia', 'Desegokia'], title='Normalizatu gabeko konfusio matrizea')

plt.show()

## Feature importance

In [ ]:
df_raw = pd.concat([train, test])
df_raw.shape

In [ ]:
fi = rf_feat_importance(df=train, m=m)

fi[:30].plot('cols', 'imp', 'barh', figsize=(20,7))
plt.show()

df_important = df_raw[fi[:30].cols]

## Check redundancy between features

In [ ]:
from scipy.cluster import hierarchy as hc

corr = np.round(scipy.stats.spearmanr(df_important).correlation, 4)
corr_condensed = hc.distance.squareform(1-corr)
z = hc.linkage(corr_condensed, method='average')
fig = plt.figure(figsize=(16,10))
dendrogram = hc.dendrogram(z, labels=df_important.columns, orientation='left', leaf_font_size=16)
plt.show()

### Here we can see that many important features are very highly correlated, so we will try to run the classifier without those who are redundant (only deleting one of both redundant cols)

In [ ]:
redundants = ['Zone12_Pressure_Time_numpy.median', 'Zone16_Pressure_Time_scipy.stats.kurtosis', 'Zone24_Pressure_Time_scipy.stats.kurtosis',
            'Zone24_tfilling_Time_numpy.median','Zone24_tfilling_Time_numpy.min', 'Zone24_tfilling_Time_numpy.mean',
             'Zone24_tfilling_Time_numpy.max', 'Zone24_tfilling_Time_numpy.percentile_95']


for red in redundants:
    m = RandomForestClassifier(n_estimators=80,max_features=0.5, min_samples_leaf=10, oob_score=True, n_jobs=-1)
    wo_red = df_important.drop(axis=1, columns=[red])
    m.fit(wo_red[:2000], target[:2000].values.ravel())
    print(f'{red}:\n')
    print(f'\tTraining score: {m.score(wo_red[:2000], target[:2000].values.ravel())}')
    print(f'\tTesting score: {m.score(wo_red[2000:], target[2000:].values.ravel())}')
    print(f'\tOut of bag score: {m.oob_score_}\n')

m = RandomForestClassifier(n_estimators=80,max_features=0.5, min_samples_leaf=10, oob_score=True, n_jobs=-1)
df_wo_redundant = df_important.drop(axis=1, columns=redundants)
m.fit(df_wo_redundant[:2000], target[:2000].values.ravel())

print(f'Without redundants:\n')
print(f'\tTraining score: {m.score(df_wo_redundant[:2000], target[:2000].values.ravel())}')
print(f'\tTesting score: {m.score(df_wo_redundant[2000:], target[2000:].values.ravel())}')
print(f'\tOut of bag score: {m.oob_score_}\n')

base_accuracy = evaluate(m, df_wo_redundant[2000:], target[2000:].values.ravel())

### We can see that without those redundant features, our model is still quite solid, so we remove them

## GridSearch in order to calculate best hiperparameters

In [ ]:
def plot_grid_search(cv_results, grid_param_1, grid_param_2, name_param_1, name_param_2):
    # Get Test Scores Mean and std for each grid search
    scores_mean = cv_results['mean_test_score']
    scores_mean = np.array(scores_mean).reshape(len(grid_param_2),len(grid_param_1))

    scores_sd = cv_results['std_test_score']
    scores_sd = np.array(scores_sd).reshape(len(grid_param_2),len(grid_param_1))

    # Plot Grid search scores
    figure, ax = plt.subplots(1,1, figsize=(15,6))

    # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
    for idx, val in enumerate(grid_param_2):
        ax.plot(grid_param_1, scores_mean[idx,:], '-o', label= name_param_2 + ': ' + str(val))

    ax.set_title("Grid Search Scores", fontsize=18, fontweight='bold')
    ax.set_xlabel(name_param_1, fontsize=16)
    ax.set_ylabel('CV Average Score', fontsize=16)
    ax.legend(loc="best", fontsize=10)
    ax.grid(True)


In [ ]:
n_estimators = [80, 100, 160, 200]
max_features = [0.5, 'log2', 'sqrt', 'auto']
grid_search = GridSearchCV(m,
            dict(n_estimators=n_estimators,
                 max_features=max_features),
                 cv=5, n_jobs=8)

grid_search.fit(df_wo_redundant[:2000], target[:2000].values.ravel());
grid_search.best_params_

In [ ]:
plot_grid_search(grid_search.cv_results_, n_estimators, max_features, 'N Estimators', 'Max Features')

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, df_wo_redundant[2000:], target[2000:])

### So, we see how the score of the max_features=0.5 is the best, and is quite constant when it comes to n_estimators. Now, we choose max_features=0.5, and we try to 

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [0.5],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300]
}
clf = RandomForestClassifier()

pipe_grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=4, n_jobs=-1, verbose=2)

pipe_grid.fit(df_wo_redundant[:2000], target[:2000].values.ravel());

In [ ]:
best_grid = pipe_grid.best_estimator_
grid_accuracy = evaluate(best_grid, df_wo_redundant[2000:], target[2000:])

## We check the how the feature importance is now and the redundat features

In [ ]:
fi = rf_feat_importance(df=df_wo_redundant[:2000], m=best_grid)

fi.plot('cols', 'imp', 'barh', figsize=(20,7))
plt.show()


In [ ]:
from scipy.cluster import hierarchy as hc

corr = np.round(scipy.stats.spearmanr(df_wo_redundant).correlation, 4)
corr_condensed = hc.distance.squareform(1-corr)
z = hc.linkage(corr_condensed, method='average')
fig = plt.figure(figsize=(16,10))
dendrogram = hc.dendrogram(z, labels=df_wo_redundant.columns, orientation='left', leaf_font_size=16)
plt.show()